In [ ]:
import os
import json
import requests


def get_textonly_response(prompt, API_KEY):
    """Get a response from the OpenAI API using only text

    Args:
        prompt (str): The prompt to use
        API_KEY (str): Your OpenAI API key
    Returns:
        response_json: The response from the OpenAI API as a JSON object
    """
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}"
    }

    payload = {
        "model": "gpt-4o",
        "top_p": 0.5,
        "messages": [
            {
             "role": "user",
             "content": [
                {
                 "type": "text",
                 "text": prompt
                }
             ]
            }
        ],
        "max_tokens": 300
    }

    response = requests.post("https://api.openai.com/v1/chat/completions",
                             headers=headers, json=payload)

    response_json = response.json()
    return response_json

def clean_response(response):
    return response['choices'][0]['message']['content']

def process_dataset(dataset_path, API_KEY, output_path, num_attempts,
                    image_dir=None):
    """Process a dataset of prompts and images using the OpenAI API

    Args:
        dataset_path (str): A list of dictionaries, each containing an 'id',
                            'prompt', and 'image'
        API_KEY (str): Your OpenAI API key
        output_path (str): The path to save the output JSON file
        num_attempts (int): The number of attempts to make for each prompt
        image_dir (str): The directory containing the images (if any)
    """
    with open(dataset_path, "r") as f:
        dataset = json.load(f)

    if not os.path.exists(output_path):
        with open(output_path, "w") as f:
            json.dump([], f)
        output_data = []

    for i in range(len(dataset)):
        id = dataset[i]['id']
        prompt = dataset[i]['prompt']
        image = dataset[i]['image']

        if image_dir is not None:
            image = f"{image_dir}/{image}"

        for j in range(num_attempts):
            if image is None:
                response = get_textonly_response(prompt, API_KEY)
            else:
                response = get_image_response(prompt, image, API_KEY)
            response = clean_response(response)
            output_json = {
                "id": id,
                "attempt": j,
                "prompt": prompt,
                "response": response
            }
            output_data.append(output_json)

    with open(output_path, "w") as f:
        json.dump(output_data, f, indent=4)

    return output_data


In [ ]:
dataset_path = "SFRT_textonly_dataset.json"
API_KEY = '' #replace with your key
num_attempts = 1 #specify the number of trial
output_path = "SFRT_output.json"


In [ ]:
output = process_dataset(dataset_path, API_KEY, output_path, num_attempts)
print("Processing complete. Output saved to", output_path)

Processing complete. Output saved to SFRT_output.json
